## Primary key / Foreign key

- https://docs.databricks.com/en/tables/constraints.html
- https://docs.databricks.com/en/sql/language-manual/sql-ref-syntax-ddl-create-table-constraint.html

In [0]:
%sql

use catalog prasad_kona_dev;
use schema default;
CREATE TABLE T(pk1 INTEGER NOT NULL, pk2 INTEGER NOT NULL,
                CONSTRAINT t_pk PRIMARY KEY(pk1, pk2));
CREATE TABLE S(pk INTEGER NOT NULL PRIMARY KEY,
                fk1 INTEGER, fk2 INTEGER,
                CONSTRAINT s_t_fk FOREIGN KEY(fk1, fk2) REFERENCES T);


In [0]:
%sql

use catalog prasad_kona_dev;
use schema default;
-- Create a table with a primary key
 CREATE or replace TABLE persons(first_name STRING NOT NULL, last_name STRING NOT NULL, nickname STRING,
                       CONSTRAINT persons_pk PRIMARY KEY(first_name, last_name));

-- create a table with a foreign key
 CREATE  or replace TABLE pets(name STRING, owner_first_name STRING, owner_last_name STRING,
                    CONSTRAINT pets_persons_fk FOREIGN KEY (owner_first_name, owner_last_name) REFERENCES persons);

-- Create a table with a single column primary key and system generated name
CREATE  or replace TABLE customers(customerid STRING NOT NULL PRIMARY KEY, name STRING);

-- Create a table with a names single column primary key and a named single column foreign key
 CREATE  or replace TABLE orders(orderid BIGINT NOT NULL CONSTRAINT orders_pk PRIMARY KEY,
                      customerid STRING CONSTRAINT orders_customers_fk REFERENCES customers);


In [0]:
%sql
use catalog prasad_kona_dev;
use schema default;

drop table if exists persons2;
drop table if exists pets2;

drop table if exists customers2;
drop table if exists orders2;

In [0]:
%sql

use catalog prasad_kona_dev;
use schema default;

--- Example 1 ---
-- Create a table with a primary key
 CREATE  or replace TABLE persons2(first_name STRING NOT NULL, last_name STRING NOT NULL, nickname STRING,
                       CONSTRAINT persons2_pk PRIMARY KEY(first_name, last_name));

-- create a table with a foreign key
 CREATE  or replace TABLE pets2(name STRING, owner_first_name STRING, owner_last_name STRING,
                    CONSTRAINT pets2_persons2_fk FOREIGN KEY (owner_first_name, owner_last_name) REFERENCES persons2 (first_name,last_name));


--- Example 2 ---
-- Create a table with a single column primary key and system generated name
CREATE  or replace TABLE customers2(customerid STRING NOT NULL PRIMARY KEY, name STRING);

-- Create a table with a names single column primary key and a named single column foreign key
 CREATE  or replace TABLE orders2(orderid BIGINT NOT NULL CONSTRAINT orders2_pk PRIMARY KEY,
                      customerid STRING CONSTRAINT orders2_customers2_fk REFERENCES customers2(customerid));


In [0]:
%sql
describe formatted prasad_kona_dev.default.pets2

col_name,data_type,comment
name,string,null
owner_first_name,string,null
owner_last_name,string,null
,,
# Detailed Table Information,,
Catalog,prasad_kona_dev,
Database,default,
Table,pets2,
Created Time,Wed Oct 09 18:18:10 UTC 2024,
Last Access,UNKNOWN,


In [0]:
%sql
describe detail prasad_kona_dev.default.pets2

format,id,name,description,location,createdAt,lastModified,partitionColumns,clusteringColumns,numFiles,sizeInBytes,properties,minReaderVersion,minWriterVersion,tableFeatures,statistics
delta,9d3c84ea-e9ff-477e-9bee-7ecb9ae6cfff,prasad_kona_dev.default.pets2,null,s3://databricks-e2demofieldengwest/b169b504-4c54-49f2-bc3a-adf4b128f36d/tables/b3162fc3-0fc0-4d14-83fb-34c33d220f63,2024-10-09T18:18:09.232Z,2024-10-09T18:18:10Z,List(),List(),0,0,Map(delta.enableDeletionVectors -> true),3,7,List(deletionVectors),"Map(numRowsDeletedByDeletionVectors -> 0, numDeletionVectors -> 0)"


In [0]:
%sql
-- Query to list all foreign key relationships for a specific table in the specified schema/catalog

use catalog prasad_kona_dev; -- Replace with your actual catalog name

select
  tc.constraint_type,
  rc.constraint_name,
  rc.unique_constraint_name,
  kcu_fk.table_catalog as child_catalog_name,
  kcu_fk.table_schema as child_schema_name,
  kcu_fk.table_name as parent_table_name,
  kcu_fk.column_name as parent_table_column_name,
  kcu_fk.ordinal_position,
  kcu_fk.position_in_unique_constraint,
  kcu_pk.table_catalog as parent_catalog_name,
  kcu_pk.table_schema as parent_schema_name,
  kcu_pk.table_name as parent_table_name,
  kcu_pk.column_name as parent_table_column_name
from
  information_schema.referential_constraints rc
  join information_schema.key_column_usage kcu_fk on rc.constraint_name = kcu_fk.constraint_name
  join information_schema.key_column_usage kcu_pk on rc.unique_constraint_name = kcu_pk.constraint_name
  and kcu_fk.position_in_unique_constraint = kcu_pk.ordinal_position
  join information_schema.table_constraints tc on rc.constraint_name = tc.constraint_name

WHERE kcu_fk.table_schema = 'default' -- Specify the schema name here
    AND kcu_fk.table_name ='pets2' -- Replace with your actual table name
    AND tc.constraint_type = 'FOREIGN KEY'
ORDER BY kcu_fk.ordinal_position asc, kcu_fk.table_catalog, kcu_fk.table_schema, kcu_fk.table_name, kcu_fk.column_name;




constraint_type,constraint_name,unique_constraint_name,child_catalog_name,child_schema_name,parent_table_name,parent_table_column_name,ordinal_position,position_in_unique_constraint,parent_catalog_name,parent_schema_name,parent_table_name,parent_table_column_name
FOREIGN KEY,pets2_persons2_fk,persons2_pk,prasad_kona_dev,default,pets2,owner_first_name,1,1,prasad_kona_dev,default,persons2,first_name
FOREIGN KEY,pets2_persons2_fk,persons2_pk,prasad_kona_dev,default,pets2,owner_last_name,2,2,prasad_kona_dev,default,persons2,last_name


In [0]:
%sql

-- Query to list primary key constraints for a specific table in the specified schema/catalog

use catalog prasad_kona_dev; -- Replace with your actual catalog name

select
  tc.constraint_type as constraint_type,
  tc.constraint_name as constraint_name,
  tc.table_catalog as catalog_name,
  tc.table_schema as schema_name,
  tc.table_name as table_name,
  kcu.column_name as column_name,
  kcu.ordinal_position as ordinal_position
from
  information_schema.table_constraints tc
  join information_schema.key_column_usage kcu on tc.constraint_name = kcu.constraint_name
WHERE tc.table_schema = 'default' -- Specify the schema name here
    AND tc.table_name ='persons2' -- Replace with your actual table name
   AND tc.constraint_type = 'PRIMARY KEY'
ORDER BY kcu.ordinal_position asc, tc.table_catalog, tc.table_schema, tc.table_name, kcu.column_name;

constraint_type,constraint_name,catalog_name,schema_name,table_name,column_name,ordinal_position
PRIMARY KEY,persons2_pk,prasad_kona_dev,default,persons2,first_name,1
PRIMARY KEY,persons2_pk,prasad_kona_dev,default,persons2,last_name,2


In [0]:
%sql
use catalog prasad_kona_dev;
use schema default;
insert into persons2 values ('f1','l1','nickname1'), ('f2','l2','nickname2');

num_affected_rows,num_inserted_rows
2,2


In [0]:
%sql
use catalog prasad_kona_dev;
use schema default;
insert into pets2 values ('pet1','f1','l1'), ('pet2','f3','l3');

num_affected_rows,num_inserted_rows
2,2


In [0]:
%sql
SELECT p2.*, pt2.name AS pet_name
FROM prasad_kona_dev.default.persons2 p2
JOIN prasad_kona_dev.default.pets2 pt2
  ON p2.first_name = pt2.owner_first_name 
  AND p2.last_name = pt2.owner_last_name;

first_name,last_name,nickname,pet_name
f1,l1,nickname1,pet1


Get table properties using rest api

In [0]:
import requests

def get_table_properties(full_table_name):
    access_token = dbutils.secrets.get(scope="prasad_kona", key="databricks_user_token")
    databricks_hostname = "e2-demo-field-eng.cloud.databricks.com"
    response = requests.get(f"https://{databricks_hostname}/api/2.1/unity-catalog/tables/{full_table_name}",
                            headers={"Authorization": f"Bearer {access_token}"})
    table_properties = response.json()
    return table_properties



example showing extracting foreign key metadata

In [0]:
# Example usage
full_table_name = "prasad_kona_dev.default.pets2"
table_properties = get_table_properties(full_table_name)
print(table_properties)

{'name': 'pets2', 'catalog_name': 'prasad_kona_dev', 'schema_name': 'default', 'table_type': 'MANAGED', 'data_source_format': 'DELTA', 'columns': [{'name': 'name', 'type_text': 'string', 'type_name': 'STRING', 'position': 0, 'type_precision': 0, 'type_scale': 0, 'type_json': '{"name":"name","type":"string","nullable":true,"metadata":{}}', 'nullable': True}, {'name': 'owner_first_name', 'type_text': 'string', 'type_name': 'STRING', 'position': 1, 'type_precision': 0, 'type_scale': 0, 'type_json': '{"name":"owner_first_name","type":"string","nullable":true,"metadata":{}}', 'nullable': True}, {'name': 'owner_last_name', 'type_text': 'string', 'type_name': 'STRING', 'position': 2, 'type_precision': 0, 'type_scale': 0, 'type_json': '{"name":"owner_last_name","type":"string","nullable":true,"metadata":{}}', 'nullable': True}], 'storage_location': 's3://databricks-e2demofieldengwest/b169b504-4c54-49f2-bc3a-adf4b128f36d/tables/b3162fc3-0fc0-4d14-83fb-34c33d220f63', 'owner': 'prasad.kona@databr

In [0]:
# Extracting table_constraints
table_constraints = table_properties['table_constraints']
print(table_constraints)

[{'foreign_key_constraint': {'name': 'pets2_persons2_fk', 'child_columns': ['owner_first_name', 'owner_last_name'], 'parent_table': 'prasad_kona_dev.default.persons2', 'parent_columns': ['first_name', 'last_name']}, 'constraint_id': '52ae41fa-a446-4e30-a4cc-80b4f103a608', 'table_id': 'b3162fc3-0fc0-4d14-83fb-34c33d220f63'}]


example showing extracting primary key metadata

In [0]:
# Example usage
full_table_name = "prasad_kona_dev.default.persons2"
table_properties = get_table_properties(full_table_name)
print(table_properties)

{'name': 'persons2', 'catalog_name': 'prasad_kona_dev', 'schema_name': 'default', 'table_type': 'MANAGED', 'data_source_format': 'DELTA', 'columns': [{'name': 'first_name', 'type_text': 'string', 'type_name': 'STRING', 'position': 0, 'type_precision': 0, 'type_scale': 0, 'type_json': '{"name":"first_name","type":"string","nullable":false,"metadata":{}}', 'nullable': False}, {'name': 'last_name', 'type_text': 'string', 'type_name': 'STRING', 'position': 1, 'type_precision': 0, 'type_scale': 0, 'type_json': '{"name":"last_name","type":"string","nullable":false,"metadata":{}}', 'nullable': False}, {'name': 'nickname', 'type_text': 'string', 'type_name': 'STRING', 'position': 2, 'type_precision': 0, 'type_scale': 0, 'type_json': '{"name":"nickname","type":"string","nullable":true,"metadata":{}}', 'nullable': True}], 'storage_location': 's3://databricks-e2demofieldengwest/b169b504-4c54-49f2-bc3a-adf4b128f36d/tables/6f0dc1eb-9c4c-46f7-a96f-8d1e76a3f610', 'owner': 'prasad.kona@databricks.com'

In [0]:
# Extracting table_constraints
table_constraints = table_properties['table_constraints']
print(table_constraints)

[{'primary_key_constraint': {'name': 'persons2_pk', 'child_columns': ['first_name', 'last_name']}, 'constraint_id': 'e405f448-cbaa-46d6-84de-07252335f764', 'table_id': '6f0dc1eb-9c4c-46f7-a96f-8d1e76a3f610'}]
